# Исследование оттока клиентов


В данной работе нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. 
Цель работы - построить модель с предельно большим значением *F1*-меры, не менее 0.59 

Применены методы кодирования данных, вычисление метрик F1 и auc_roc.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

Признаки
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак
- Exited — факт ухода клиента

# 1. Подготовка данных

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.utils import shuffle

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
df = pd.read_csv('/datasets/Churn.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:

df.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


В столбце Tenue есть пропуски, их около 10% от всего датасета, если их удалить вряд ли это сильно изменит результат обучения, поэтому удалим пропуски оттуда.

In [5]:
df.dropna(subset = ['Tenure'], inplace=True)
df['Tenure'] = df['Tenure'].astype('int')


In [6]:
#убедились, что все наны удалены
df['Tenure'].value_counts()

1     952
2     950
8     933
3     928
5     927
7     925
4     885
9     882
6     881
10    446
0     382
Name: Tenure, dtype: int64

In [7]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,15719294,Wood,800,France,Female,29,2,0.00,2,0,0,167773.55,0
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1


Нужно удалить столбцы, котрые не будут участвовать в создании модели:

In [8]:
#useless_features = ['RowNumber', 'CustomerId', 'Surname']
#df = df.drop(useless_features, axis=1)
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

Теперь закодируем датасет, чтобы столбцы с  категориальными признаками, стали численными и потом разделим датасет на три выборки обучающая - 60%, валидационная - 20% и тестовая - 20%.

In [9]:
df_ohe = pd.get_dummies(df, drop_first=True)
target = df_ohe['Exited']
features = df_ohe.drop('Exited', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, random_state=12345)

# 2. Исследование задачи

In [10]:
df['Exited'].value_counts()

0    7237
1    1854
Name: Exited, dtype: int64

Классы несбалансированны, соотношение не 1:1.Примерное распределение это 80 и 20%.

Построим модели случайного леса и решающего дерева на основе несбалансированных данных и выстроим гиперпараметры. Оценим значения метрики F1 и auc roc score.

In [11]:
#случайный лес
for estim in range(50, 120, 10):
    model = RandomForestClassifier(n_estimators=estim, random_state=12345, max_depth=10)
    model.fit(features_train, target_train)
    #model.fit(features_valid, target_valid)
    
    #predictions_test = model.predict(features_test)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    print("f1 =", estim, ":", f1)

f1 = 50 : 0.5722222222222223
f1 = 60 : 0.5730337078651686
f1 = 70 : 0.57703081232493
f1 = 80 : 0.5633802816901409
f1 = 90 : 0.5633802816901409
f1 = 100 : 0.5754189944134078
f1 = 110 : 0.573816155988858


In [12]:
probabilities_valid1 = model.predict_proba(features_valid)
probabilities_one_valid1 = probabilities_valid1[:, 1]


auc_roc1 = roc_auc_score(target_valid, probabilities_one_valid1)
print(auc_roc1)

0.8672761301468471


Значение метрики F1 оптималтно при n_estimators=100 и глубине дерева 10. 

In [13]:
#решающее дерево
for depth in range(2, 15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    #model.fit(features_valid, target_valid)
    
    #predictions_test = model.predict(features_test)
    predicted_valid = model.predict(features_valid)
    f1_2 = f1_score(target_valid, predicted_valid)
    print("n_depth =", depth, ":", f1_2)
    

n_depth = 2 : 0.5643564356435643
n_depth = 3 : 0.44012944983818775
n_depth = 4 : 0.5730027548209367
n_depth = 5 : 0.5212121212121212
n_depth = 6 : 0.5754189944134078
n_depth = 7 : 0.5729166666666667
n_depth = 8 : 0.5464190981432361
n_depth = 9 : 0.5343511450381679
n_depth = 10 : 0.4961636828644501
n_depth = 11 : 0.5206812652068126
n_depth = 12 : 0.4964200477326968
n_depth = 13 : 0.5069767441860464
n_depth = 14 : 0.48291571753986334


In [235]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]


auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.67852001151742


В результате выберем модель случайного леса, так как метрики F1 и  roc_auc_score имеют наибольшие значения

# 3. Борьба с дисбалансом

Поборемся с дисбалансом классов с помощью атрибута class_weight.

In [14]:
model_balanced = DecisionTreeClassifier(random_state=12345, max_depth=6, class_weight = 'balanced')
model_balanced.fit(features_valid, target_valid)

predicted_valid = model_balanced.predict(features_valid)
f1_balanced = f1_score(target_valid, predicted_valid)
print("F1:", f1_balanced)

F1: 0.6699029126213593


In [15]:
model_balanced1 = RandomForestClassifier(n_estimators=240, random_state=12345, max_depth=10, class_weight = 'balanced')
model_balanced1.fit(features_valid, target_valid)

predicted_valid1 = model_balanced1.predict(features_valid)
f1_balanced1 = f1_score(target_valid, predicted_valid1)
print("F1:", f1_balanced1)

F1: 0.9891540130151845


Для модели случайного леса получилось достичь значения 0.98, значит будем тестировать именно эту модель на тестовой выборке, но сначала проверим еще способ балансировки.

Применим метод увеличения выборки:

In [16]:
def upsample(features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    return features_upsampled, target_upsampled
    

features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]

repeat = 5
features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
# < добавьте перемешивание >


features_upsampled, target_upsampled = upsample(features_train, target_train, 5)


In [17]:
model_balanced2 = DecisionTreeClassifier(random_state=12345, max_depth=6)
model_balanced2.fit(features_upsampled, target_upsampled)

predicted_valid2 = model_balanced2.predict(features_valid)
f1_balanced2 = f1_score(target_valid, predicted_valid2)
print("F1:", f1_balanced2)

F1: 0.5650224215246638


In [18]:
model_balanced3 = RandomForestClassifier(n_estimators=240, random_state=12345, max_depth=10)
model_balanced3.fit(features_upsampled, target_upsampled)

predicted_valid3 = model_balanced3.predict(features_valid)
f1_balanced3 = f1_score(target_valid, predicted_valid3)
print("F1:", f1_balanced3)

F1: 0.6070175438596491


Значение метрики F1 для случайного леса 0.61, нам этого достаточно, так что ее и проверим на тестовых данных.

# 4. Тестирование модели

Теперь обучим модель случайного леса и протестируем  с успешными гиперпараметрами n_estimators=100 и max_depth=10, уравновешенную по классам.

In [241]:
model = RandomForestClassifier(n_estimators=100, random_state=12345, max_depth=10,  class_weight = 'balanced')
model.fit(features_train, target_train)

predicted_test = model.predict(features_test)
f1 = f1_score(target_test, predicted_test)
print("F1:", f1)

F1: 0.6013071895424836


In [242]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]


auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8649006622516556


Значение метрики F1 соответствует заявленному минимуму 0.59 в задании, так же значение auc_roc не на много отличается от значения при проверке на валидационной выборке.

В результате работы мы создали модель машинного обучения, которая дает достаточный уровень F1. Для этого  в первичном  датасете были найдены пропуски и обработаны с учетом их кол-ва, было решено их удалить, так как отсутствие 10% данным не привело бы к ухудшению качества модели, а заполнение этих пропусков средним значением или медианой возможно привело бы к переобучению модели. Далее были удалены столбцы, не участвующие в работе модели. Затем данные были закодированы для того, чтобы столбцы с категориальными данными стали числовыми. И потом весь исходный датасет был разделен на обучающую - 60%, валтдационную - 20% и тестовую - 20% выборки. Решено было создать две модели: дерева решений и случайного леса. В начале создаем модель и обучаем ее на обучающей выборке, далее настраиваем гиперпараметры и проверяем точность на валидационной. 
После оценки дисбаланса было выявлено, что он присутствует, примерно в соотношении 80 на 20%. Далее провели тестирование моделей с выбранными гиперпараметрами и с проведенным балансом классов двумя способами : увеличением выборки и class_weight. На основе всех полученных значений метрик выбрали модел для тестирования на тестовой выборке.
Для проверки на тестовой выборке я решила использовать модель случайного леса, так как на ней в заданном мной диапазоне гиперпараметров значения метрик были больше, чем у другой модели. Результаты проверки модели на тестовой выборке показали, что модель обладает значением F1 0.60, что соответствует заявленным пороговым значениям  в задании, а так же метрика auc_roc имеет значение 0.86, что близко к значению этой же метрики при проверке на валидационной выборке.